## Sentence Quality Score (SQS)

$$ SQS(s) = \frac{\sum_{i, j \in workers(s)} Wcos(WorkVec(i,s), WorkVec(j,s)) \; WQS(i) \; WQS(j)}{\sum_{i, j \in workers(s)} WQS(i) \; WQS(j)}, \; i \neq j .$$


### Weighted Cosine

$$ Wcos(vec_1, vec_2) = \frac{\sum_{r} vec_1(r) \; vec_2(r) \; RQS(r)}{\sqrt{(\sum_{r} vec_1^2(r) \; RQS(r)) \; (\sum_{r} vec_2^2(r) \; RQS(r))}} .$$ 


## Worker Quality Score (WQS)

$$ WQS(i) = WSA(i) \; WWA(i) .$$

### Worker-Sentence Agreement

$$ WSA(i) = \frac{\sum_{s \in sentences(i)} Wcos(WorkVec(i,s), SentVec(s) - WorkVec(i, s)) \; SQS(s)}{\sum_{s \in sentences(i)} SQS(s)} .$$

### Worker-Worker Agreement

$$ WWA(i) = \frac{ \sum_{j \in workers(s \in sentences(i))} Wcos(WorkVec(i, s), WorkVec(j, s)) \; WQS(j) \; SQS(s) }{ \sum_{j \in workers(s \in sentences(i))} WQS(j) \; SQS(s) }, \; i \neq j .$$


## Relation Quality Score (RQS)

$$ RQS(r) = \frac{ \sum_{i,j} WQS(i) \; WQS(j) \; P_r(i | j) }{ \sum_{i,j} WQS(i) \; WQS(j) }, i \neq j .$$

$$ P_r(i | j) = \frac{ \sum_{s \in sentences(i) \cap sentences(j) } SQS(s) \; WorkVec(i, s)(r) \; WorkVec(j, s)(r) }{ \sum_{s \in sentences(i) \cap sentences(j) } SQS(s) \; WorkVec(j, s)(r) } . $$


## Sentence-Relation Score (SRS)

$$ SRS(s) = \frac{ \sum_{i \in workers(s)} WorkVec(i,s) \; WQS(i) }{ \sum_{i \in workers(s)} WQS(i) }. $$